# Google Drive をマウントする

In [1]:
# 必要なライブラリを読み込む
from google.colab import drive 

# Google ドライブをマウントする
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


必要なモジュールをインポートします。

*   pandas：表データの前処理・集計に便利な型「データフレーム」を提供してくれるライブラリ
*   mlxtend：機械学習用のライブラリです。sklearn とはまた違った機能が実装されています。
> *   frequent_patterns：頻度に関する分析を行う際に便利なモジュールがいくつか用意されています。今回はアソシエーション分析用モジュール apriori と見つかったルールを表示するためのメソッド association_rules をインポートします。

In [2]:
# pandas のインポート
import pandas as pd

# アソシエーション分析用のモジュールをインポート
from mlxtend.frequent_patterns import apriori,association_rules

# データの読み込み

In [6]:
# 買物データを読み込む
OnlineRetail = pd.read_csv("drive/My Drive/DATA-SCI-TECH-EXERCISE/DATA/OnlineRetail.csv")
OnlineRetail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010/12/1 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010/12/1 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010/12/1 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010/12/1 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010/12/1 8:26,3.39,17850.0,United Kingdom


In [7]:
# 概要を確認する
OnlineRetail.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


# データの準備
* IDと商品を抽出する

In [8]:
# 利用する変数を取得
columnList = ['CustomerID','StockCode']
X = OnlineRetail.loc[:,columnList]
# 購入有無だけを知りたいので「2件」などは不要
X = X.drop_duplicates()

In [9]:
# ユーザー（横）と商品（縦）で購入有無を確認したデータを作成する
X_count_tmp = X.groupby(['CustomerID','StockCode']).size().reset_index(name='Count')
X_count = (X_count_tmp.groupby(['CustomerID','StockCode'])['Count'].sum().unstack().reset_index().fillna(0).set_index('CustomerID'))
X_count.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,15030,15034,15036,15039,15044A,15044B,15044C,15044D,15056BL,15056N,15056P,15058A,15058B,15058C,15060B,16008,16010,16011,16012,16014,16015,16016,16020C,16033,16043,16045,16046,16048,16049,16052,...,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# アソシエーション分析（ア・プリオリアルゴリズム）
*   min_support：抽出するルールの支持度（support）の下限値を指定します。
*   use_colname：True にしておくと、元データのカラム名を利用してくれます。
*   以下は重要な指標です。
>   * Support（支持度）：アイテムがデータ内に出現する頻度を表します。
>   * Confidence（確信度）：if（先行節）-then（帰結節）文が正しいと認められる頻度を表します。
>   * Lift（リフト）：確信度が予想よりどの程度高いか、または低いかを表します。Liftが1.0より大きい場合、アイテム間の相関が予想よりも有意であることを意味します。Liftが大きいほど相関性が高いことを意味します。

In [10]:
# アプリオリアルゴリズムによるアソシエーション分析を実施
frequent_itemsets = apriori(X_count, min_support=0.05, use_colnames=True)

In [11]:
# リフト（lift）が1以上のルールを抽出する
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
# ルールを表示する（リフトの降順）
rules.sort_values(by = 'lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
245,"(22698, 22699)","(22697, 22423)",0.062443,0.068618,0.051693,0.827839,12.064371,0.047408,5.409939
240,"(22697, 22423)","(22698, 22699)",0.068618,0.062443,0.051693,0.753333,12.064371,0.047408,3.800907
248,(22698),"(22697, 22423, 22699)",0.073422,0.059927,0.051693,0.704050,11.748496,0.047293,3.176458
237,"(22697, 22423, 22699)",(22698),0.059927,0.073422,0.051693,0.862595,11.748496,0.047293,6.743430
243,"(22423, 22698)","(22697, 22699)",0.059698,0.074108,0.051693,0.865900,11.684310,0.047268,6.904509
...,...,...,...,...,...,...,...,...,...
180,(85123A),(84879),0.196249,0.155306,0.054437,0.277389,1.786076,0.023959,1.168947
187,(85099B),(85123A),0.145471,0.196249,0.050778,0.349057,1.778643,0.022229,1.234748
186,(85123A),(85099B),0.196249,0.145471,0.050778,0.258741,1.778643,0.022229,1.152808
104,(85123A),(22423),0.196249,0.202882,0.059927,0.305361,1.505118,0.020111,1.147529


In [12]:
# 確信度（confidence）が1以上のルールを抽出する
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
# ルールを表示する（確信度の降順）
rules.sort_values(by = 'confidence',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
150,"(22423, 22698, 22699)",(22697),0.053522,0.089204,0.051693,0.965812,10.827000,0.046918,26.640782
143,"(22698, 22699)",(22697),0.062443,0.089204,0.060156,0.963370,10.799624,0.054585,24.864730
128,"(22423, 22698)",(22697),0.059698,0.089204,0.056496,0.946360,10.608940,0.051171,16.979839
77,(22698),(22697),0.073422,0.089204,0.068161,0.928349,10.407029,0.061612,12.711544
147,"(22697, 22423, 22698)",(22699),0.056496,0.097667,0.051693,0.914980,9.368364,0.046175,10.613155
...,...,...,...,...,...,...,...,...,...
54,(23209),(22382),0.107274,0.112306,0.055124,0.513859,4.575545,0.043076,1.826003
15,(20725),(85099B),0.121683,0.145471,0.062443,0.513158,3.527557,0.044741,1.755248
59,(22384),(23209),0.101784,0.107274,0.052150,0.512360,4.776196,0.041231,1.830706
37,(23209),(20728),0.107274,0.110018,0.054437,0.507463,4.612530,0.042635,1.806933
